In [1]:
import pandas as pd
import pickle
from sklearn.svm import LinearSVC 

In [2]:
group_data_path = "group_final_res/b0f84ebd"

In [3]:
group_train_path = "{}/train_data.pkl".format(group_data_path)
group_val_path = "{}/val_data.pkl".format(group_data_path)

In [4]:
with open(group_train_path, 'rb') as f:
    group_train = pickle.load(f)

Using backend: pytorch


In [5]:
with open(group_val_path, 'rb') as f:
    group_val = pickle.load(f)

In [6]:
with open("{}/model-36.pkl".format(group_data_path), 'rb') as f:
    best_group_model = pickle.load(f)

In [7]:
best_group_model.test(group_val)

0.9

In [9]:
group_train[0][0].ndata["feat"].shape

torch.Size([2375, 74])

In [10]:
group_train[0][1]

1

In [13]:
import numpy as np
import torch

In [16]:
torch.mean(group_train[0][0].ndata["feat"], axis=0).numpy()

array([ 2.8108296e-09,  4.0154706e-09, -6.0232064e-10, -4.4170179e-09,
       -4.0154706e-09, -8.0309415e-10,  5.6216591e-09, -3.6139238e-09,
       -1.6061883e-09,  6.4247532e-09,  4.0154707e-10,  1.6061883e-09,
        2.8108296e-09,  1.8069619e-09, -4.4170179e-09, -8.0309412e-09,
        8.0309415e-10,  8.0309415e-10, -1.7065751e-09, -7.2278477e-09,
        1.2046413e-09,  1.4054148e-09,  8.8340357e-09,  8.0309415e-10,
        6.4247532e-09, -2.8108296e-09,  0.0000000e+00, -2.4092826e-09,
       -3.2123766e-09, -8.0309415e-10,  8.0309412e-09,  6.0232064e-10,
       -4.8185651e-09,  2.4092826e-09,  8.0309415e-10,  8.0309412e-09,
        8.0309415e-10,  0.0000000e+00, -8.0309415e-10, -4.0154706e-09,
       -1.6061883e-09,  4.0154706e-09, -4.8185651e-09,  0.0000000e+00,
        5.6216591e-09,  3.2123766e-09, -3.2123766e-09,  5.6216591e-09,
        0.0000000e+00, -8.0309412e-09,  8.0309415e-10, -8.0309415e-10,
       -5.2201119e-09, -1.6061883e-09, -6.4247532e-09, -8.0309415e-10,
      

In [54]:
svm_train_data_group = [[i[0].ndata["feat"].numpy(), i[1]] for i in group_train]

In [55]:
svm_val_data_group = [[i[0].ndata["feat"].numpy(), i[1]] for i in group_val]

In [61]:
svm_train_data_group = [[i[0].ndata["feat"].numpy(), np.array([i[1]]*i[0].ndata['feat'].shape[0])] for i in group_train]

In [63]:
X_train = np.concatenate([i[0] for i in svm_train_data_group])

In [64]:
y_train = np.concatenate([i[1] for i in svm_train_data_group])

In [89]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()


In [90]:
model.fit(X_train, y_train)

RandomForestClassifier()

In [91]:
model_preds_raw_group = [model.predict(i[0]) for i in svm_val_data_group]

In [92]:
round(sum(model_preds_raw_group[0])/len(model_preds_raw_group[0]))

1

In [93]:
group_maj_vote = [round(sum(i)/len(i)) for i in model_preds_raw_group]

In [95]:
def accuracy(first, second):
    assert(len(first) == len(second))
    corr = 0
    for i in range(len(first)):
        if first[i] == second[i]:
            corr+=1
    return corr/len(first)

In [97]:
accuracy(group_maj_vote, [i[1] for i in svm_val_data_group])

0.6428571428571429

In [98]:
stage_data_path = "stage_final_res/fa73b31b"

In [99]:
stage_train_path = "{}/train_data.pkl".format(stage_data_path)
stage_val_path = "{}/val_data.pkl".format(stage_data_path)

In [100]:
with open(stage_train_path, 'rb') as f:
    stage_train = pickle.load(f)

In [101]:
with open(stage_val_path, 'rb') as f:
    stage_val = pickle.load(f)

In [102]:
with open("{}/model-48.pkl".format(stage_data_path), 'rb') as f:
    best_stage_model = pickle.load(f)

In [103]:
best_stage_model.test(stage_val)

0.83

In [115]:
model = RandomForestClassifier()

In [109]:
svm_train_data_stage = [[i[0].ndata["feat"].numpy(), np.array([i[1]]*i[0].ndata['feat'].shape[0])] for i in stage_train]

In [110]:
svm_val_data_stage = [[i[0].ndata["feat"].numpy(), i[1]] for i in stage_val]

In [111]:
X_train = np.concatenate([i[0] for i in svm_train_data_stage])

In [112]:
y_train = np.concatenate([i[1] for i in svm_train_data_stage])

In [116]:
model.fit(X_train, y_train)

RandomForestClassifier()

In [117]:
model_preds_raw_stage = [model.predict(i[0]) for i in svm_val_data_stage]

In [118]:
stage_maj_vote = [round(sum(i)/len(i)) for i in model_preds_raw_stage]

In [120]:
accuracy(stage_maj_vote, [i[1] for i in svm_val_data_stage])

0.8095238095238095

In [123]:
stage_gnn_res = best_stage_model.infer([i[0] for i in stage_val])

In [126]:
def precision(predicted, actual):
    assert(len(predicted) == len(actual))
    all_positive = 0
    true_positive = 0
    for i in range(len(predicted)):
        if predicted[i] == 1:
            all_positive+=1
            if actual[i] == 1:
                true_positive+=1
    return true_positive/all_positive

In [127]:
def recall(predicted, actual):
    assert(len(predicted) == len(actual))
    actual_positives = 0
    retrived_positives = 0
    for i in range(len(predicted)):
        if actual[i] == 1:
            actual_positives+=1
            if predicted[i] == 1:
                retrived_positives+=1
    return retrived_positives/actual_positives

In [128]:
precision(stage_maj_vote, [i[1] for i in svm_val_data_stage])

0.0

In [129]:
precision(stage_gnn_res, [i[1] for i in svm_val_data_stage])

0.5

In [133]:
recall(stage_maj_vote, [i[1] for i in svm_val_data_stage])

0.0

In [134]:
recall(stage_gnn_res, [i[1] for i in svm_val_data_stage])

0.5714285714285714

In [135]:
sum([i[1] for i in svm_val_data_stage])

7